# Lean4 Demo: LLM-Assisted Theorem Proving

**Lean4** is a theorem prover and programming language. Unlike Dafny (which verifies imperative code), Lean proves mathematical theorems using tactics.

In this notebook, we'll:
1. See what Lean4 proofs look like
2. Ask an LLM to fill in `sorry` placeholders (incomplete proofs)
3. Run the verification loop with Lean's type checker

## Why Lean4?

Lean4 is the hottest formal verification language right now:
- [Harmonic AI](https://harmonic.fun/) raised $120M building on Lean4
- DeepMind's AlphaProof uses Lean4
- [Lean Copilot](https://arxiv.org/abs/2404.12534) showed LLMs can automate 74% of proof steps

The key insight: **LLM hallucinations don't matter** because Lean's type checker rejects invalid proofs.

In [ ]:
import sys
sys.path.insert(0, '..')

from src.llm_client import LLMClient
from src.verifiers import LeanVerifier
from rich import print as rprint
from rich.syntax import Syntax
from rich.panel import Panel

client = LLMClient()

try:
    verifier = LeanVerifier()
    print("✅ Client and Lean verifier ready")
except RuntimeError as e:
    print(f"⚠️ Lean not installed: {e}")
    print("Run ./setup.sh to install Lean4")
    verifier = None

## Lean4 Proof Structure

A Lean4 proof typically looks like:

```lean
theorem name : statement := by
  tactic1
  tactic2
  ...
```

Common tactics:
- `intro` - introduce hypothesis
- `apply` - apply a lemma
- `exact` - provide exact term
- `simp` - simplification
- `omega` - linear arithmetic
- `rfl` - reflexivity

The `sorry` tactic is a placeholder for incomplete proofs—Lean accepts it but marks the theorem as unproven.

## Example 1: Simple Arithmetic Theorems

Let's start with theorems an LLM can prove in 1-2 tactics.

In [ ]:
# Theorems with sorry placeholders
theorems_to_prove = [
    {
        "name": "add_comm",
        "statement": "theorem add_comm_example : ∀ a b : Nat, a + b = b + a := by\n  sorry",
        "hint": "Use omega or Nat.add_comm"
    },
    {
        "name": "add_zero", 
        "statement": "theorem add_zero_right : ∀ n : Nat, n + 0 = n := by\n  sorry",
        "hint": "Use intro then rfl or simp"
    },
    {
        "name": "imp_self",
        "statement": "theorem imp_self : ∀ P : Prop, P → P := by\n  sorry",
        "hint": "Use intro twice"
    },
    {
        "name": "and_left",
        "statement": "theorem and_left : ∀ P Q : Prop, P ∧ Q → P := by\n  sorry",
        "hint": "Use intro and And.left or cases"
    },
]

for t in theorems_to_prove:
    rprint(Panel(Syntax(t['statement'], 'lean', theme='monokai'), 
                 title=f"Theorem: {t['name']}", subtitle=t['hint']))

In [ ]:
def prove_theorem(statement: str, max_attempts: int = 3):
    """Ask LLM to prove a Lean theorem."""
    if verifier is None:
        print("⚠️ Lean not available, showing LLM output only")
    
    for attempt in range(1, max_attempts + 1):
        print(f"\n--- Attempt {attempt}/{max_attempts} ---")
        
        # Ask LLM for proof
        response = client.generate_lean_proof(statement)
        proof = client._extract_code(response.content)
        
        print(f"Tokens used: {response.input_tokens + response.output_tokens}")
        rprint(Panel(Syntax(proof, 'lean', theme='monokai'), title="LLM's Proof"))
        
        if verifier is None:
            return proof, attempt
        
        # Verify
        result = verifier.verify(proof)
        
        if result.success:
            print("✅ Proof verified!")
            return proof, attempt
        else:
            print(f"❌ Verification failed: {result.error}")
            # Update statement with error context for next attempt
            statement = f"{statement}\n\n-- Previous attempt failed with: {result.error}"
    
    print(f"⚠️ Failed after {max_attempts} attempts")
    return proof, max_attempts

In [ ]:
# Try proving the first theorem
proof, attempts = prove_theorem(theorems_to_prove[0]['statement'])

In [ ]:
# Try the implication theorem
proof, attempts = prove_theorem(theorems_to_prove[2]['statement'])

## Comparison with Dafny

| Aspect | Dafny | Lean4 |
|--------|-------|-------|
| Style | Annotated imperative code | Mathematical proofs |
| LLM task | Add invariants/contracts | Generate tactics |
| Feedback | Line-specific errors | Type errors, unsolved goals |
| Best for | Program verification | Math theorems, type theory |

## Key Insight: Lean Copilot Results

From the [Lean Copilot paper](https://arxiv.org/abs/2404.12534):

> "When assisting humans, Lean Copilot requires only 2.08 manually-entered proof steps on average (compared to 3.86 required by aesop); when automating the theorem proving process, Lean Copilot automates 74.2% of proof steps."

The key is the **feedback loop**: Lean tells you exactly which goal is unsolved, and the LLM can iterate.

## Next: TLA+

Continue to [04-tlaplus-demo.ipynb](04-tlaplus-demo.ipynb) to see model checking with TLA+.